In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Set Variables

In [4]:
height = 32
width = 32
channels = 3
batch_size = 32
epochs = 20
num_classes = 10

# Function

In [5]:
def PreprocessData(X_Train, Y_Train, X_Test, Y_Test, nums_class):
    #Normalize picture
    X_Train = X_Train.astype('float32')/255.
    X_Test = X_Test.astype('float32')/255.
    #split training data into training and validation set
    X_train, X_valid, Y_train, Y_valid = train_test_split(X_Train, Y_Train, test_size=0.2, random_state=7, shuffle= True)
    Y_train = keras.utils.to_categorical(Y_train)
    Y_valid = keras.utils.to_categorical(Y_valid)
    Y_Test = keras.utils.to_categorical(Y_Test)
    return ((X_train, Y_train), (X_valid, Y_valid), (X_Test, Y_Test))

In [6]:
def Aug_Data(X_train, Y_train, X_valid, Y_valid, batch_size):
    train_datagen = keras.preprocessing.image.ImageDataGenerator(
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    )
    validation_datagen = keras.preprocessing.image.ImageDataGenerator()
    train_generator = train_datagen.flow(X_train, Y_train, batch_size=batch_size)
    valid_generator = validation_datagen.flow(X_valid, Y_valid, batch_size=batch_size)
    return train_generator, valid_generator

In [7]:
def BuildModel(filter_number, layer_number, dropout, optimization):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(filters=filter_number, 
                                  kernel_size=(3,3), 
                                  padding='same', 
                                  activation='relu',
                                  input_shape=[width, height, channels]))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(dropout)),
    model.add(keras.layers.MaxPool2D(pool_size=2))
    for i in range(layer_number):
        model.add(keras.layers.Conv2D(filters=filter_number*(2**(i+1)), 
                                  kernel_size=(3,3), 
                                  padding='same', 
                                  activation='relu',
                                  input_shape=[width, height, channels]))
        model.add(keras.layers.BatchNormalization())
        keras.layers.Dropout(dropout)
        model.add(keras.layers.MaxPool2D(pool_size=2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(256))
    model.add(keras.layers.ReLU())
    model.add(keras.layers.Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimization, metrics=['acc'])
    return model

# Preprocess Data

In [8]:
cifar10 = keras.datasets.cifar10.load_data()
(x_train_all, y_train_all), (x_test, y_test) = cifar10
(X_train, Y_train), (X_valid, Y_valid), (X_test, Y_test) = PreprocessData(x_train_all, y_train_all, x_test, y_test, num_classes)
train_generator, valid_generator = Aug_Data(X_train, Y_train, X_valid, Y_valid, batch_size)
print(X_train.shape)

(40000, 32, 32, 3)


In [ ]:
# model = keras.models.Sequential()
# model.add(keras.layers.Conv2D(64,(3,3),input_shape=[width, height, channels], padding="same"))
# model.add(keras.layers.Activation('relu'))
# model.add(keras.layers.BatchNormalization())    
# model.add(keras.layers.MaxPool2D(pool_size=(2,2)))    

# model.add(keras.layers.Conv2D(128,(3,3),input_shape=[width, height, channels], padding="same"))
# model.add(keras.layers.Activation('relu'))
# model.add(keras.layers.BatchNormalization())   
# model.add(keras.layers.MaxPool2D(pool_size=(2,2)))

# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(256))
# model.add(keras.layers.Activation('relu'))
# model.add(keras.layers.Dropout(0.5))

# model.add(keras.layers.Dense(num_classes))
# model.add(keras.layers.Activation('softmax'))

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
# model.summary()

In [ ]:
# model = keras.models.Sequential([
#     keras.layers.Conv2D(filters=32, 
#                         kernel_size=3, 
#                         padding='same', 
#                         input_shape=[width, height, channels]),
#     keras.layers.LeakyReLU(),
#     keras.layers.Conv2D(filters=32, 
#                         kernel_size=3, 
#                         padding='same'),
#     keras.layers.LeakyReLU(),
#     keras.layers.Dropout(0.25),
#     keras.layers.MaxPool2D(pool_size=(2,2)),
#     keras.layers.Conv2D(filters=64, 
#                         kernel_size=3, 
#                         padding='same'),
#     keras.layers.LeakyReLU(),
#     keras.layers.Conv2D(filters=64, 
#                         kernel_size=3, 
#                         padding='same'),
#     keras.layers.LeakyReLU(),
#     keras.layers.Dropout(0.25),
#     keras.layers.MaxPool2D(pool_size=(2,2)),
#     keras.layers.Conv2D(filters=128, 
#                         kernel_size=3, 
#                         padding='same'),
#     keras.layers.LeakyReLU(),
#     keras.layers.Conv2D(filters=128, 
#                         kernel_size=3, 
#                         padding='same'),
#     keras.layers.LeakyReLU(),
#     keras.layers.Dropout(0.25),
#     keras.layers.MaxPool2D(pool_size=(2,2)),
#     keras.layers.Flatten(),
#     keras.layers.Dropout(0.25),
#     keras.layers.Dense(256),
#     keras.layers.LeakyReLU(),
#     keras.layers.Dense(num_classes, activation='softmax'), 
# ])

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

# Build Model

In [ ]:
model = BuildModel(32, 2, 0.25, 'adam')
model.summary()

# Training Process

In [ ]:
earlystop = keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True, verbose=1)
lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_delta=0.0001)
callbacks = [earlystop, lr]
history = model.fit(train_generator,
                    steps_per_epoch = len(train_generator) // batch_size,
                    epochs = epochs,
                    callbacks = callbacks,
                    validation_data = valid_generator,
                    validation_steps = len(valid_generator) // batch_size)

In [3]:
model = keras.models.load_model('kaggle/best_model.h5')

In [9]:
model.evaluate(X_test, Y_test)

313/313 [==============================] - 4s 12ms/step - loss: 0.6928 - acc: 0.7787


[0.6927950978279114, 0.7786999940872192]

In [ ]:
def plots_learning_curves(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
plots_learning_curves(history, 'accuracy', epochs, 0, 1)
plots_learning_curves(history, 'loss', epochs, 0, 2.5)

In [ ]:
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory='./',
                                                    x_col='filepath',
                                                    y_col='class',
                                                    classes=class_names,
                                                    target_size = (height, width), 
                                                    batch_size=batch_size,
                                                    seed = 7,
                                                    shuffle = False,
                                                    class_mode='sparse')

test_num = test_generator.samples
print(test_num)

In [ ]:
test_predict = model.predict_generator(test_generator,
                                       workers = 10,
                                       use_multiprocessing = True)